In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats


In [ ]:
df = pd.read_csv('heart.csv')

In [ ]:
data_types = {
    'Количественный непрерывный': ['age', 'trestbps', 'chol', 'thalach', 'oldpeak'],
    'Количественный дискретный': ['ca'],
    'Категориальный (бинарный)': ['sex', 'fbs', 'exang', 'target'],
    'Категориальный/ранговый': ['cp', 'restecg', 'slope', 'thal']
}
quant_features = data_types['Количественный непрерывный'] + data_types['Количественный дискретный']
rank_features = data_types['Категориальный/ранговый']

In [ ]:
print("\nСтатистическое описание датасета (количественные признаки):")
print(df[quant_features].describe().T)

In [ ]:
cols = 3
rows = 5
fig, axes = plt.subplots(rows, cols, figsize=(15, 5 * rows))
axes = axes.flatten()

for i, col in enumerate(df.columns):
    sns.histplot(df[col], bins=30, kde=True, ax=axes[i])
    axes[i].set_title(f'Распределение {col}')

for j in range(i + 1, len(axes)):
    fig.delaxes(axes[j])

plt.tight_layout()
plt.show()

In [ ]:
sns.pairplot(df[['age', 'chol', 'cp', 'target']])

In [ ]:
# Корреляция Пирсона для количественных непрерывных нормальных признаков
continuous_features = ['age', 'trestbps', 'chol', 'thalach']
pearson_corr = df[continuous_features].corr(method='pearson')
print(pearson_corr)

# Визуализация корреляции Пирсона
plt.figure(figsize=(10, 8))
sns.heatmap(pearson_corr, annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Корреляция Пирсона между количественными непрерывными нормальными признаками')
plt.tight_layout()
plt.show()

In [ ]:
# Корреляция Спирмена для всех количественных признаков (не требует нормальности)
spearman_corr = df[quant_features].corr(method='spearman')
print(spearman_corr)

# Визуализация корреляции Спирмена
plt.figure(figsize=(10, 8))
sns.heatmap(spearman_corr, annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Корреляция Спирмена между количественными признаками')
plt.tight_layout()
plt.show()

In [ ]:
# Корреляция Спирмена для всех ранговых признаков
spearman_corr = df[rank_features].corr(method='spearman')
print(spearman_corr)

# Визуализация корреляции Спирмена
plt.figure(figsize=(10, 8))
sns.heatmap(spearman_corr, annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Корреляция Спирмена между ранговыми признаками')
plt.tight_layout()
plt.show()

In [ ]:
# Корреляция Спирмена между ранговыми и количественными признаками
spearman_corr = df[rank_features + quant_features].corr(method='spearman')
print(spearman_corr)

# Визуализация корреляции Спирмена
plt.figure(figsize=(10, 8))
sns.heatmap(spearman_corr, annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Корреляция Спирмена между ранговыми и количественными признаками')
plt.tight_layout()
plt.show()

In [ ]:
# Корреляция Кендалла для всех количественных признаков
kendall_corr = df[quant_features].corr(method='kendall')
print(kendall_corr)

# Визуализация корреляции Кендалла
plt.figure(figsize=(10, 8))
sns.heatmap(kendall_corr, annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Корреляция Кендалла для всех количественных признаков')
plt.tight_layout()
plt.show()

In [ ]:
# Point-biserial корреляция между количественными и бинарными признаками
for quant in quant_features:
    r, p = stats.pointbiserialr(df[quant], df['target'])
    print("%10s - target" % (quant) + f": r = {round(r, 4)}, p-value = {round(p, 3)}")

In [ ]:
# Хи-квадрат для категориальных признаков
for cat in data_types['Категориальный (бинарный)'] + data_types['Категориальный/ранговый']:
    if cat == 'target':
        continue

    contingency_table = pd.crosstab(df[cat], df['target'])
    chi2, p, dof, expected = stats.chi2_contingency(contingency_table)
    n = contingency_table.sum().sum()
    v = np.sqrt(chi2 / (n * min(contingency_table.shape) - 1))

    print(f"{cat} - target: Chi2 = {round(chi2, 4)}, p-value = {round(p, 3)}, V Крамера = {round(v, 4)}")

    # Вывод таблицы сопряженности
    print(f"Таблица сопряженности {cat} и target:")
    print(contingency_table)
    print()

In [ ]:
# Для количественных признаков с целевой переменной - ящик с усами
plt.figure(figsize=(15, 10))
for i, quant in enumerate(quant_features):
    plt.subplot(2, 3, i + 1)
    sns.boxplot(x='target', y=quant, data=df)
    plt.title(f'{quant} vs target')
plt.tight_layout()
plt.show()